In [75]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import copy

By Lauren Schmutz (contact: lschmutz@andrew.cmu.edu). Pushed to github July 14, 2023.
Notes:
1.  Model does not experience significant efficiency drawbacks. Should run well with
    100+ periods; but arbitrage concerns exist. Note the arbitrage detector builtin.
2.  Class structure in a barebones state. No inheritance is used.
    Instead a "wrapper" model class stores the stock and interest models as
    a base, and a bond class is enabled to calculated two-coin price via method.
    This way, multiple bonds can be calculated and compared using the same stock
    and rate model, and new models may be initialized as well.
3.  Check input variables and sample numbers for how to use the model. For "call list",
    an input of None means no call option at that period. Otherwise, use a numerical
    input to represent the strike price at that time. For convertible and distress lists,
    input 0 means the stock is not convertible/never in distress at that period. Note
    further that these inputs expect the same number of elements in the list as there are
    periods in the model (including 0, i.e. t+1), even though having such optionality at time 0
    may not make sense. There are no failsafe mechanisms that prevent users
    from these inputs, though, and a user will encounter errors if they use them
    improperly. In the distress list, the program expects tuples of
    (distress price, coupon multiplier) at all periods except the final one, which expects
    (distress price, coupon multiplier, face multiplier).

In [76]:
def displayChart(tab, cap=None):
    Chart=[]
    columns = []
    N = len(tab)-1 if cap==None else cap-1
    for i in range(N, -1, -1):
        chart = []
        columns.append(N-i)
        for j in range(0,i):
            chart.append("")
        for j in range(i, N+1):
            entry = tab[j][i]
            chart.append(entry)
        Chart.append(chart)
    df = pd.DataFrame(Chart, columns = columns)
    return df.style.hide_index()

In [77]:
def list_to_dict(L):
    res = {}
    for i in range(len(L)):
        res[i] = L[i]
    return res

In [78]:
def tree(t, init, up, down, negative_allowed):
    """t=num of time periods. init=initial value. up, down factors : num, negative values allowed : bool."""
    res = {}
    gap = up+down
    if negative_allowed:
        for i in range(t+1):
            res[i] = np.arange(init-i*down, init+(i+1)*up, gap)
    else:
        for i in range(t+1):
            res[i] = np.maximum(np.zeros(i+1), np.arange(init-i*down, init+(i+1)*up, gap))
    return res

In [79]:
# def bond_tree(bond):
#     print("unimplemented")
#     L = []
#     for i in range(bond.periods):
#         pass

In [80]:
class HoLee:
    def __init__(self, periods, R0, lam, sig):
        self.periods = periods
        self.R0 = R0
        self.lam = lam
        self.sig = sig
        self.tree_exists = False

    def create_tree(self):
        if self.lam+self.sig > 0:
            self.up = self.lam+self.sig
            self.down = self.sig-self.lam
        else:
            self.up = self.sig-self.lam
            self.down = self.lam+self.sig
        
        self.tree = tree(self.periods, self.R0, self.up, self.down, True)
        self.tree_exists = True
        return self.tree

In [81]:
class Stocks:
    def __init__(self, periods, S0, alpha, beta):
        self.periods = periods
        self.S0 = S0
        self.alpha = alpha
        self.up = self.alpha
        self.beta = beta
        self.down = self.beta
        self.tree_exists = False

    def create_tree(self):
        self.tree = tree(self.periods, self.S0, self.alpha, self.beta, False)
        self.tree_exists = True
        return self.tree

In [82]:
class Model:
    """Model contains objects from HoLee, Stocks. Creation of trees are delayed via methods to address potential efficiency concerns."""
    def __init__(self, periods):
        self.periods = periods
        self.stocks_exist = False
        self.rates_exist = False
        self.arbitrage = False
        self.arbitrage_cells = {}
    
    def init_stocks(self, S0, alpha, beta):
        self.stocks_obj = Stocks(self.periods, S0, alpha, beta)
        self.stock_up = alpha
        self.stock_down = beta
        self.stocks_tree = self.stocks_obj.create_tree()
        self.stocks_exist = True

    def init_HoLee(self, R0, lam, sig):
        self.rates_obj = HoLee(self.periods, R0, lam, sig)
        if lam+sig > 0:
            self.rate_up = lam+sig
            self.rate_down = sig-lam
        else:
            self.rate_up = sig-lam
            self.rate_down = lam+sig
        self.rates_tree = self.rates_obj.create_tree()
        self.rates_exist = True

In [83]:
def rate_p_values(model):
    t = model.periods
    res = {}
    for i in range(t):
        res[i] = np.full(i+1, 0.5)
    res[t] = np.ones(t+1)
    # model.rate_p_vals = res
    return res

In [84]:
def p_values(model):
    t = model.periods
    if not (model.stocks_exist and model.rates_exist):
            print("need to initialize stocks and/or rates")
            return None
    stock_price_chart = model.stocks_tree
    rates = model.rates_tree
    P = {i: np.ones(i+1) for i in range (t+1)}
    # P = {i: [1]*(i+1) for i in range(t+1)}
    for i in range(0, t):
        for j in range(i+1):
            if stock_price_chart[i][j] == 0:
                P[i][j] = -1
            else:
                d = stock_price_chart[i+1][j] / stock_price_chart[i][j]
                u = stock_price_chart[i+1][j+1] / stock_price_chart[i][j]
                p_soon = (1 + rates[i][j] - d) / (u-d)
                if (p_soon<0) or (p_soon>1):
                    model.arbitrage = True
                    model.arbitrage_cells[(i,j)] = (model.stocks_obj.tree[i][j], u, d, model.rates_obj.tree[i][j], p_soon)
                P[i][j] = p_soon
    return P

In [85]:
class Bond:
    def __init__(self, model, face, q, distr_list, conv_list, call_list):
        self.model = model
        self.periods = self.model.periods
        self.face = face
        self.q = q
        self.coup = self.face * self.q
        self.distr_dict = list_to_dict(distr_list)
        self.conv_dict = list_to_dict(conv_list)
        self.call_dict = list_to_dict(call_list)
        self.p_vals_exist = False

    def init_p_vals(self):
        if not (self.model.stocks_exist or self.model.rates_exist):
            print("need to initialize stocks and rates in outer model. do so and try again")
            return False
        elif not self.model.stocks_exist:
            print("need to initialize stocks in outer model. do so and try again")
            return False
        elif not self.model.rates_exist:
            print("need to initialize rates in outer model. do so and try again")
            return False
        else:
            self.p_vals = p_values(self.model)
            self.rate_p_vals = rate_p_values(self.model)
            if self.model.arbitrage: print("WARNING: Arbitrage exists in model.")
            return True
    
    def print_arbitrages(self):    
        for key in self.model.arbitrage_cells:
                print(str(key)+': ', end='') # S, u, d, r, p
                print("S={}, u={}, d={}, r={}, p={}".format(*self.model.arbitrage_cells[key]))
    
    def one_coin(self):
        print("unimplemented")
        if not self.init_p_vals():
            return False
        face = self.face
        t = self.model.periods
        S = self.model.stocks_tree
        R = self.model.rates_tree
        S_p = self.p_vals
        R_p = self.rate_p_vals
        coup = self.coup
        res = {}
        return None


# Idea: one dictionary. Key: integer tuple (time period, # stock heads, # tails), Value: bond price before coupon.
#                                                                             or: value: [bond price b4 coup, coup at time]
# WARNING: call formula not currently implemented

    def two_coin(self):
        if not self.init_p_vals():
            return False
        # self.p_vals = p_values(self.model)
        # self.rate_p_vals = rate_p_values(self.model)
        face = self.face
        t = self.model.periods
        S = self.model.stocks_tree
        R = self.model.rates_tree
        S_p = self.p_vals
        R_p = self.rate_p_vals
        coup = self.coup
        res = {}
        #fill in the last period 
        cur_call = self.call_dict[t]
        for i in range(t+1):            #i: number of stock heads
            for j in range(t+1):        #j: number of rate heads
                cur_stock = S[t][i]
                res[(t, i, j)] = [0,0]
                if cur_stock > 0:
                    cur_exp = face * (self.distr_dict[t][2] if cur_stock <= self.distr_dict[t][0] else 1)
                    if cur_call != None:
                        cur_exp = min(cur_exp, cur_call)
                    res[(t, i, j)][0] = max(self.conv_dict[t]*cur_stock, cur_exp)
                res[(t, i, j)][1] = coup if cur_stock > self.distr_dict[t][0] else (self.distr_dict[t][1] * coup if cur_stock > 0 else 0)
                # res[(t, i, j)] = max(conv_list[t]*cur_stock, face * (bond.distr_list[t+1][2] if cur_stock <= bond.distr_list[t+1][0] else 1))
        #now all the final period prices have been filled. what remains is to perform backwards induction
        #Backwards Induction:
        for n in range(t-1, -1, -1):
            cur_call = self.call_dict[n]
            for i in range(n+1):        #i: number of stock heads
                for j in range(n+1):    #j: number of rate heads
                    cur_stock = S[n][i]
                    cur_rate = R[n][j]
                    cur_disc = 1/(1+cur_rate)
                    #p values
                    s_h_p = S_p[n][i]
                    s_t_p = 1 - s_h_p
                    r_h_p = R_p[n][j]
                    r_t_p = 1 - r_h_p
                    #there are four cases. two adjacent stock price moves and two adjacent rates moves. make sure to get the correct pvals
                    #stock head rate head
                    hh = res[(n+1, i+1, j+1)][0]
                    hh_c = res[(n+1, i+1, j+1)][1]
                    #stock head rate tail
                    ht = res[(n+1, i+1, j)][0]
                    ht_c = res[(n+1, i+1, j)][1]
                    #stock tail rate head
                    th = res[(n+1, i, j+1)][0]
                    th_c = res[(n+1, i, j+1)][1]
                    #stock tail rate tail
                    tt = res[(n+1, i, j)][0]
                    tt_c = res[(n+1, i, j)][1]
                    #fill formula
                    cur_exp = 0 if cur_stock == 0 else cur_disc * (s_h_p*r_h_p*(hh+hh_c) + s_h_p*r_t_p*(ht+ht_c) + s_t_p*r_h_p*(th+th_c) + s_t_p*r_t_p*(tt+tt_c))
                    #call
                    if cur_call != None:
                        cur_exp = min(cur_exp, cur_call)
                    res[(n, i, j)] = [0,0]
                    res[(n, i, j)][0] = cur_exp
                    res[(n, i, j)][1] = coup if cur_stock > self.distr_dict[n][0] else (self.distr_dict[n][1] * coup if cur_stock > 0 else 0)
                    if n == 0: res[(n, i, j)][1] = 0
        if self.model.arbitrage:
            print("WARNING: Arbitrage exists in model at {} cells. To display them call self.print_arbitrages()".format(len(self.model.arbitrage_cells)))
            
        return res


In [86]:
t_global = 48
model = Model(t_global)
alpha_monthly = 10 # up amount if we were to have 12 periods
beta_monthly = 10 # down amount if we were to have 12 periods
model.init_stocks(30,alpha_monthly*12/t_global,beta_monthly*12/t_global)
print("stocks")
displayChart(model.stocks_tree, cap=12)

stocks


/var/folders/v2/8kmltg8j0d1glmqdfvfvmprc0000gn/T/ipykernel_72664/4127606966.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4,5,6,7,8,9,10,11
,,,,,,,,,,,57.500000
,,,,,,,,,,55.000000,52.500000
,,,,,,,,,52.500000,50.000000,47.500000
,,,,,,,,50.000000,47.500000,45.000000,42.500000
,,,,,,,47.500000,45.000000,42.500000,40.000000,37.500000
,,,,,,45.000000,42.500000,40.000000,37.500000,35.000000,32.500000
,,,,,42.500000,40.000000,37.500000,35.000000,32.500000,30.000000,27.500000
,,,,40.000000,37.500000,35.000000,32.500000,30.000000,27.500000,25.000000,22.500000
,,,37.500000,35.000000,32.500000,30.000000,27.500000,25.000000,22.500000,20.000000,17.500000
,,35.000000,32.500000,30.000000,27.500000,25.000000,22.500000,20.000000,17.500000,15.000000,12.500000


In [87]:
sigma_monthly = 0.001
lambda_monthly = 0
model.init_HoLee(0.04, lambda_monthly*1/t_global, sigma_monthly*1/t_global)
print("interest rates")
displayChart(model.rates_tree, 12)

interest rates


/var/folders/v2/8kmltg8j0d1glmqdfvfvmprc0000gn/T/ipykernel_72664/4127606966.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4,5,6,7,8,9,10,11
,,,,,,,,,,,0.040229
,,,,,,,,,,0.040208,0.040188
,,,,,,,,,0.040188,0.040167,0.040146
,,,,,,,,0.040167,0.040146,0.040125,0.040104
,,,,,,,0.040146,0.040125,0.040104,0.040083,0.040063
,,,,,,0.040125,0.040104,0.040083,0.040063,0.040042,0.040021
,,,,,0.040104,0.040083,0.040063,0.040042,0.040021,0.040000,0.039979
,,,,0.040083,0.040063,0.040042,0.040021,0.040000,0.039979,0.039958,0.039938
,,,0.040063,0.040042,0.040021,0.040000,0.039979,0.039958,0.039938,0.039917,0.039896
,,0.040042,0.040021,0.040000,0.039979,0.039958,0.039938,0.039917,0.039896,0.039875,0.039854


In [88]:
distr_list = [(10, 0) for i in range(t_global + 1)]
distr_list[t_global] = (10, 0, 0.5)
conv_list = np.ones(t_global+1)
call_list = np.full(t_global+1, None)
cbond1 = Bond(model, 50, 0.04, distr_list, conv_list, call_list)
cbond1.init_p_vals()
print("p tilde")
displayChart(cbond1.p_vals, cap=12)

p tilde


/var/folders/v2/8kmltg8j0d1glmqdfvfvmprc0000gn/T/ipykernel_72664/4127606966.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4,5,6,7,8,9,10,11
,,,,,,,,,,,0.962635
,,,,,,,,,,0.942292,0.921969
,,,,,,,,,0.921969,0.901667,0.881385
,,,,,,,,0.901667,0.881385,0.861125,0.840885
,,,,,,,0.881385,0.861125,0.840885,0.820667,0.800469
,,,,,,0.861125,0.840885,0.820667,0.800469,0.780292,0.760135
,,,,,0.840885,0.820667,0.800469,0.780292,0.760135,0.740000,0.719885
,,,,0.820667,0.800469,0.780292,0.760135,0.740000,0.719885,0.699792,0.679719
,,,0.800469,0.780292,0.760135,0.740000,0.719885,0.699792,0.679719,0.659667,0.639635
,,0.780292,0.760135,0.740000,0.719885,0.699792,0.679719,0.659667,0.639635,0.619625,0.599635


In [89]:
print("rate p tilde")
displayChart(cbond1.rate_p_vals, cap=12)

rate p tilde


/var/folders/v2/8kmltg8j0d1glmqdfvfvmprc0000gn/T/ipykernel_72664/4127606966.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  return df.style.hide_index()


0,1,2,3,4,5,6,7,8,9,10,11
,,,,,,,,,,,0.500000
,,,,,,,,,,0.500000,0.500000
,,,,,,,,,0.500000,0.500000,0.500000
,,,,,,,,0.500000,0.500000,0.500000,0.500000
,,,,,,,0.500000,0.500000,0.500000,0.500000,0.500000
,,,,,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
,,,,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
,,,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
,,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000


In [90]:
D = cbond1.two_coin()
print(D[(0,0,0)])

[73.08187327687217, 0]


Arbitrage will exist in many iterations of the model even with different parameters because as the stock prices increases, the constant alpha and beta are not large enough to make 1+r < u (u decreases towards 1 and d increases towards 1). To fix this we either need a multiplicative model or to make it so that the program checks if increasing the rate would bring it above u-1 and prevent arbitrage in one of the following ways (or another one that we think of) 
1. Keep the rate constant or increase it to some intermediate value like (u-1+r)/2 (exact value doesn't matter so long as it is between r and u-1)
2. Guarantee that the rate decreases
3. Change the rules on how the stock price evolves so that u is always kept above 1+r
4. Some combination of changes to the interest rate evolution and the stock evolution that keeps the model arbitrage-free

Here is an equation we could solve to get around this problem. In the "best-case" scenario for both the stock and rate, the proportion of the stock price that is alpha should be greater than the interest rate. This gives at least a rudimentary condition (at least sufficient for a single-coin model where we are looking at time $n$). Let $r_n^n := r_0+\lambda n+\sigma n$ (highest possible interest rate after $n$ periods).

$$\frac{\alpha}{S_0+n\alpha} > r_n^n$$
$$\iff \alpha > r_n^n (S_0+n\alpha)$$
$$\iff \alpha > S_0 r_n^n + \alpha\cdot n r_n^n$$
$$\iff \alpha(1 - n r_n^n) > S_0 r_n^n$$

At this point note that if $r_n^n\geq1/n$ then we get that a nonpositive number is greater than the product of two strictly positive numbers (for now we assume that $r_0>0$ and thus $r_n^n>0$). Therefore, we absolutely must have that $r_n^n<1/n$, and only under that assumption can we proceed to say

$$\iff \frac{\alpha}{S_0} > \frac{r_n^n}{1 - n r_n^n}$$

In the model above we had $N=48$, $\alpha=10\cdot12/48=2.5$, $S_0=30$, $r_0=0.04$, $\lambda=0$, and $\sigma=0.001/48$. The first condition, $r_n^n<1/n$ is true for $n\leq24$, so we had arbitrage for $n$ where $n>24$ or
$$\frac{2.5}{30} = \frac{1}{12} \leq \frac{0.04+0.001n/48}{1-n(0.04+0.001n/48)} = \frac{1.92+0.001n}{48-n(1.92+0.001n)}$$
Solving this or plotting it shows that this is true for $n\geq12.834$ and indeed we see that our first arbitrage occurs at $n=13$.

In [92]:
def sim_two_coin(t, x, r, rate_vol):
    model = Model(t)
    model.init_stocks(30,x,x)
    model.init_HoLee(r, rate_vol, rate_vol)
    distr_list = [(t, 0) for i in range(t + 1)]
    distr_list[t_global] = (t, 0, 0.5)
    conv_list = np.ones(t+1)
    call_list = np.full(t+1, None)
    cbond1 = Bond(model, 50, r, distr_list, conv_list, call_list)
    cbond1.init_p_vals()
    D = cbond1.two_coin()
    return D[(0,0,0)][0]